In [ ]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()

In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [ ]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

Descarga de empresas en sistema


In [ ]:
llamadaEmpresaAntiguas = "SELECT Id,Empresa__r.Id, Empresa__r.Name, Empresa__r.Holding__c, Empresa__r.Campa_a__c, Empresa__r.Id_Empresa__c, Empresa__r.Empleador__c, Empresa__r.poliza__c, Empresa__r.RUT_Empresa__c, Empresa__r.Corredor__c,Empresa__r.Padre_Empresa__c, Name FROM Programas_por_Empresas__c WHERE Empresa__r.Padre_Empresa__c = false"
vllamadaEmpresa = sf.smart_query(llamadaEmpresaAntiguas)
vllamadaEmpresa.to_excel("F:/Empresas SF/reportes/empresas_Antiguas.xlsx", index=False)
vllamadaEmpresa.head()

vllamadaEmpresa.shape


In [ ]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  SELECT Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  FROM account where ispersonaccount = false  AND Nueva_Estructura__c = False
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

In [ ]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

In [ ]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

In [ ]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

In [ ]:
query2 = "select P_liza__c, Nombre_de_Empresa__c,Identificador__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c "

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]
df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()

fin empresas y comienzo de rescate de pacientes


In [ ]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone,PersonContactId  from Account where ispersonaccount = True 
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
        "PersonContactId",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
        "PersonContactId": "Id Contacto",
    }
)
Pacientes_SF.shape

In [ ]:
def siPrimerCero(y):
    if len(y) <= 7:
        return "NONE"
    if y[0] == "0":
        y = y[1:]
    return y

In [ ]:
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("–", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].apply(lambda y: siPrimerCero(y))
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()

In [ ]:
Pacientes_Nones1 = Pacientes_SF[(Pacientes_SF["Rut"] == "NONE")]
Pacientes_SF = Pacientes_SF[(Pacientes_SF["Rut"] != "NONE")]
Pacientes_Nones1.to_excel("F:/Empresas SF/reportes/NONE1.xlsx", index=False)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


# buscar empresas desde Paciente_Programas\_\_c relaciones de empresas con pacientes

results = sf.query_all(
"""
select Id,Paciente**c, Programas_por_Empresas**c,padre**c from Paciente_Programas**c  
 """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
columns={
"Id": "Id Pacientes_Programas**c",
"Paciente**c": "Id Account",
"Programas_por_Empresas**c": "Programas_por_Empresas**c",
"padre\_\_c": "Nombre Campaña",
}
)
Paciente_Programas_SF.shape


# Union de Pacientes Programas y Programas por empresa

empresas_unidas = pd.merge(
left=Paciente_Programas_SF,
right=Programas_empresas,
how="left",
left_on="Programas_por_Empresas**c",
right_on="Programas_por_Empresas**c",
)
empresas_unidas.shape


Carga de Archivo carga


In [368]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Instruvalve"
Pais = "Chile"
baseFinal = True
# Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo.head(5)

,rut1,rut2,nombres,Primer apellido,Segundo apellido,apellidos,email,celular,nacimiento,sexo,poliza,empleador,rutE
0,10063271-3,10063271-3,MARCOS JAVIER,ALARCON,ANDRADE,ALARCON ANDRADE,marcos.alarcon@instruvalve.cl,(+56) 982893563,1966-06-19,NaN,24277,24277,24277
1,17867250-9,17867250-9,JAVIER IGNACIO,ALARCON,SAN MARTIN,ALARCON SAN MARTIN,javier.alarcon@instruvalve.cl,(+56) 988276605,1991-04-02,NaN,33330,33330,33330
2,19430135-9,19430135-9,RODRIGO ALEJANDRO,ALARCON,SAN MARTIN,ALARCON SAN MARTIN,rodrigo.alarcon@instruvalve.com.pe,(+56) 978775171,1996-10-11,NaN,35349,35349,35349
3,26945028-2,26945028-2,JESUS RENE,GIBENS,CALANCHE,GIBENS CALANCHE,jesus.gibens@instruvalve.cl,(+56) 933037469,1954-01-07,NaN,19731,19731,19731
4,15360659-5,15360659-5,JIMMY JESUS,ORELLANA,CARRASCO,ORELLANA CARRASCO,jimmy.orellana@instruvalve.cl,(+56) 981381146,1984-04-20,NaN,30792,30792,30792


In [369]:
Pacientes_Archivo.shape

(91, 13)

In [370]:
Pacientes_IDrut = Pacientes_SF[
    [
        "Rut",
        "Id Account Paciente",
        "Id Contacto",
    ]
]

In [371]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("–", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(" ", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()

Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].astype(str)
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.upper()
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.strip()
Pacientes_Archivo.shape

Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].astype(str)
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.upper()
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace("-", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace("–", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace(".", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\2565835100.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\2565835100.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace(".", "")


(91, 14)

In [372]:
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rut2"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("-", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("–", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(" ", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.upper()
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\2296516763.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")


(91, 15)

In [403]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("┐", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("═", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("╔", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("┌", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("┐", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("═", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("╔", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("┌", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\4081690804.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\4081690804.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [374]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].apply(lambda y: siPrimerCero(y))
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].apply(lambda y: siPrimerCero(y))
Pacientes_Archivo

# Pacientes_Nones = Pacientes_Archivo[(Pacientes_Archivo["rut"] == "NONE") | (Pacientes_Archivo["rutT"] == "NONE")]
# Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE") | (Pacientes_Archivo["rutT"] != "NONE")]

,rut1,rut2,nombres,Primer apellido,Segundo apellido,apellidos,email,celular,nacimiento,sexo,poliza,empleador,rutE,rut,rutT
0,10063271-3,10063271-3,Marcos Javier,ALARCON,ANDRADE,Alarcon Andrade,marcos.alarcon@instruvalve.cl,(+56) 982893563,1966-06-19,NaN,24277,24277,24277,100632713,100632713
1,17867250-9,17867250-9,Javier Ignacio,ALARCON,SAN MARTIN,Alarcon San Martin,javier.alarcon@instruvalve.cl,(+56) 988276605,1991-04-02,NaN,33330,33330,33330,178672509,178672509
2,19430135-9,19430135-9,Rodrigo Alejandro,ALARCON,SAN MARTIN,Alarcon San Martin,rodrigo.alarcon@instruvalve.com.pe,(+56) 978775171,1996-10-11,NaN,35349,35349,35349,194301359,194301359
3,26945028-2,26945028-2,Jesus Rene,GIBENS,CALANCHE,Gibens Calanche,jesus.gibens@instruvalve.cl,(+56) 933037469,1954-01-07,NaN,19731,19731,19731,269450282,269450282
4,15360659-5,15360659-5,Jimmy Jesus,ORELLANA,CARRASCO,Orellana Carrasco,jimmy.orellana@instruvalve.cl,(+56) 981381146,1984-04-20,NaN,30792,30792,30792,153606595,153606595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,26612826-6,26.612.786-3,Marco Antonio,NaN,NaN,Palacios Chacon,NaN,NaN,2011-08-11,Masculino,40766,40766,40766,266128266,266127863
87,26693611-7,17.953.808-3,Wladimir,NaN,NaN,Carrillo Muñoz,NaN,NaN,2019-02-06,Masculino,43502,43502,43502,266936117,179538083
88,27535749-9,17.953.808-3,Maureen Alondra,NaN,NaN,Carrillo Muñoz,NaN,NaN,2021-05-14,Femenino,44330,44330,44330,275357499,179538083
89,189107897-8,17.207.594-0,Daniela,NaN,NaN,Silva Aballay,NaN,NaN,1994-12-06,Femenino,34674,34674,34674,1891078978,172075940


In [ ]:
# Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/Pacientes_Archivo22.xlsx", index=False)

In [375]:
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE")]
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rutT"] != "NONE")]
# | (Pacientes_Archivo["rutT"] != "NONE")


PAcientes Programas


In [376]:
query_pacientePrograma = (
    "SELECT Paciente__c,Programas_por_Empresas__r.Empresa__c,Paciente__r.IdentificationId__pc,Id from Paciente_Programas__c where padre__c =  '"
    + campaña
    + "'"
)
pacientePrograma = sf.smart_query(query_pacientePrograma)
pacientePrograma.head()

100%|██████████| 1/1 [00:03<00:00,  3.36s/it]


,Paciente__c,Programas_por_Empresas__r.Empresa__c,Paciente__r.IdentificationId__pc,Id
0,0018c00002amVemAAE,0018c00002cXJTOAA4,192096642,a1Y8c00000C6VYZEA3
1,0018c00002amVenAAE,0018c00002cXJTOAA4,118755383,a1Y8c00000C6VYaEAN
2,0018c00002amR77AAE,0018c00002cXJTOAA4,13256101K,a1Y8c00000C6VYbEAN
3,0018c00002amREFAA2,0018c00002cXJTOAA4,151844901,a1Y8c00000C6VYdEAN
4,0018c00002amRwrAAE,0018c00002cXJTOAA4,178672509,a1Y8c00000C6VYfEAN


In [377]:
pacientePrograma = pacientePrograma.rename(
    columns={
        "Id": "Id Paciente Programas",
        "Paciente__r.IdentificationId__pc": "Rut",
        "Programas_por_Empresas__r.Empresa__c": "Id Account Empleador",
        "Paciente__c": "Id Account Paciente",
    }
)
pacientePrograma.head()

,Id Account Paciente,Id Account Empleador,Rut,Id Paciente Programas
0,0018c00002amVemAAE,0018c00002cXJTOAA4,192096642,a1Y8c00000C6VYZEA3
1,0018c00002amVenAAE,0018c00002cXJTOAA4,118755383,a1Y8c00000C6VYaEAN
2,0018c00002amR77AAE,0018c00002cXJTOAA4,13256101K,a1Y8c00000C6VYbEAN
3,0018c00002amREFAA2,0018c00002cXJTOAA4,151844901,a1Y8c00000C6VYdEAN
4,0018c00002amRwrAAE,0018c00002cXJTOAA4,178672509,a1Y8c00000C6VYfEAN


In [378]:
pacientePrograma["Rut"] = pacientePrograma["Rut"].astype(str)
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace("-", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace("–", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].astype(str)
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace(".", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.upper()
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.strip()
pacientePrograma.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\3172767859.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace(".", "")


(80, 4)

In [379]:
PacientesProgramas_PacientesSF = pd.merge(
    left=pacientePrograma,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)

PacientesProgramas_PacientesSF.head()

,Id Account Paciente,Id Account Empleador,Rut_x,Id Paciente Programas,Rut_y,Id Contacto
0,0018c00002amVemAAE,0018c00002cXJTOAA4,192096642,a1Y8c00000C6VYZEA3,192096642,0038c000035CfKzAAK
1,0018c00002amVenAAE,0018c00002cXJTOAA4,118755383,a1Y8c00000C6VYaEAN,118755383,0038c000035CfL0AAK
2,0018c00002amR77AAE,0018c00002cXJTOAA4,13256101K,a1Y8c00000C6VYbEAN,13256101K,0038c000035CanAAAS
3,0018c00002amREFAA2,0018c00002cXJTOAA4,151844901,a1Y8c00000C6VYdEAN,151844901,0038c000035CauTAAS
4,0018c00002amRwrAAE,0018c00002cXJTOAA4,178672509,a1Y8c00000C6VYfEAN,178672509,0038c000035CbcxAAC


account contact relation


In [380]:
query_accountContact = (
    "select Id,ContactId, AccountId, RutContact__c ,Contact.AccountId from AccountContactRelation where Account.Campa_a__c = '"
    + campaña
    + "'"
)
accountContactRelation = sf.smart_query(query_accountContact)
accountContactRelation.head()

100%|██████████| 1/1 [00:03<00:00,  3.22s/it]


,Id,ContactId,AccountId,RutContact__c,Contact.AccountId
0,07k8c00000S8y5VAAR,0038c000035CfKzAAK,0018c00002cXJTOAA4,192096642,0018c00002amVemAAE
1,07k8c00000S8y5WAAR,0038c000035CfL0AAK,0018c00002cXJTOAA4,118755383,0018c00002amVenAAE
2,07k8c00000S8y5XAAR,0038c000035CanAAAS,0018c00002cXJTOAA4,13256101K,0018c00002amR77AAE
3,07k8c00000S8y5YAAR,0038c000035CbwNAAS,0018c00002cXJTOAA4,155839155,0018c00002amSGNAA2
4,07k8c00000S8y5ZAAR,0038c000035CauTAAS,0018c00002cXJTOAA4,151844901,0018c00002amREFAA2


In [381]:
accountContactRelation = accountContactRelation.rename(
    columns={
        "Id": "Id AccountContactRelation",
        "ContactId": "Id Contact Paciente",
        "AccountId": "Id Account Empresa",
        "RutContact__c": "Rut",
        "Contact.AccountId": "Id Account Paciente",
    }
)
accountContactRelation.head()

,Id AccountContactRelation,Id Contact Paciente,Id Account Empresa,Rut,Id Account Paciente
0,07k8c00000S8y5VAAR,0038c000035CfKzAAK,0018c00002cXJTOAA4,192096642,0018c00002amVemAAE
1,07k8c00000S8y5WAAR,0038c000035CfL0AAK,0018c00002cXJTOAA4,118755383,0018c00002amVenAAE
2,07k8c00000S8y5XAAR,0038c000035CanAAAS,0018c00002cXJTOAA4,13256101K,0018c00002amR77AAE
3,07k8c00000S8y5YAAR,0038c000035CbwNAAS,0018c00002cXJTOAA4,155839155,0018c00002amSGNAA2
4,07k8c00000S8y5ZAAR,0038c000035CauTAAS,0018c00002cXJTOAA4,151844901,0018c00002amREFAA2


In [382]:
accountContactRelation["Rut"] = accountContactRelation["Rut"].astype(str)
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace("-", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace("–", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].astype(str)
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace(".", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.upper()
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.strip()
accountContactRelation.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\2268980097.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace(".", "")


(81, 5)

In [383]:
AccountContact_PacientesSF = pd.merge(
    left=accountContactRelation,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)
AccountContact_PacientesSF.head()

,Id AccountContactRelation,Id Contact Paciente,Id Account Empresa,Rut_x,Id Account Paciente,Rut_y,Id Contacto
0,07k8c00000S8y5VAAR,0038c000035CfKzAAK,0018c00002cXJTOAA4,192096642,0018c00002amVemAAE,192096642,0038c000035CfKzAAK
1,07k8c00000S8y5WAAR,0038c000035CfL0AAK,0018c00002cXJTOAA4,118755383,0018c00002amVenAAE,118755383,0038c000035CfL0AAK
2,07k8c00000S8y5XAAR,0038c000035CanAAAS,0018c00002cXJTOAA4,13256101K,0018c00002amR77AAE,13256101K,0038c000035CanAAAS
3,07k8c00000S8y5YAAR,0038c000035CbwNAAS,0018c00002cXJTOAA4,155839155,0018c00002amSGNAA2,155839155,0038c000035CbwNAAS
4,07k8c00000S8y5ZAAR,0038c000035CauTAAS,0018c00002cXJTOAA4,151844901,0018c00002amREFAA2,151844901,0038c000035CauTAAS


empresa contacto convenio


In [384]:
query_empresacontactoconvenio = (
    "select Id,Paciente__r.IdentificationId__pc,  Empresa__c, Paciente__c,N_Documento_Titular__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '"
    + campaña
    + "'"
)
empresacontactoconvenio = sf.smart_query(query_empresacontactoconvenio)
empresacontactoconvenio.head()

100%|██████████| 1/1 [00:02<00:00,  2.35s/it]


,Id,Paciente__r.IdentificationId__pc,Empresa__c,Paciente__c,N_Documento_Titular__c
0,a40RN0000011frlYAA,100632713,a3yRN0000006s9DYAQ,0018c00002egmwuAAA,100632713
1,a40RN0000011frmYAA,105317476,a3yRN0000006s9DYAQ,0018c00002egmwfAAA,105317476
2,a40RN0000011frnYAA,105692226,a3yRN0000006s9DYAQ,0018c00002egmwtAAA,105692226
3,a40RN0000011froYAA,106493316,a3yRN0000006s9DYAQ,0018c00002amTYaAAM,106493316
4,a40RN0000011frpYAA,115527029,a3yRN0000006s9DYAQ,0018c00002egmwWAAQ,106493316


In [385]:
empresacontactoconvenio = empresacontactoconvenio.rename(
    columns={
        "Id": "Id EmpresaContactoConvenio",
        "Paciente__r.IdentificationId__pc": "Rut",
        "Empresa__c": "Id Convenio Empresa",
        "Paciente__c": "Id Account Paciente",
        "N_Documento_Titular__c": "Rut Titular",
    }
)
empresacontactoconvenio.head()

,Id EmpresaContactoConvenio,Rut,Id Convenio Empresa,Id Account Paciente,Rut Titular
0,a40RN0000011frlYAA,100632713,a3yRN0000006s9DYAQ,0018c00002egmwuAAA,100632713
1,a40RN0000011frmYAA,105317476,a3yRN0000006s9DYAQ,0018c00002egmwfAAA,105317476
2,a40RN0000011frnYAA,105692226,a3yRN0000006s9DYAQ,0018c00002egmwtAAA,105692226
3,a40RN0000011froYAA,106493316,a3yRN0000006s9DYAQ,0018c00002amTYaAAM,106493316
4,a40RN0000011frpYAA,115527029,a3yRN0000006s9DYAQ,0018c00002egmwWAAQ,106493316


In [386]:
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].astype(str)
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace("-", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace("–", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].astype(str)
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(".", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(" ", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.upper()
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.strip()
empresacontactoconvenio.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\2240900084.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(".", "")


(94, 5)

In [387]:
empresacontactoconvenio_PacientesSF = pd.merge(
    left=empresacontactoconvenio,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)
empresacontactoconvenio_PacientesSF.head()

,Id EmpresaContactoConvenio,Rut_x,Id Convenio Empresa,Id Account Paciente,Rut Titular,Rut_y,Id Contacto
0,a40RN0000011frlYAA,100632713,a3yRN0000006s9DYAQ,0018c00002egmwuAAA,100632713,100632713,0038c000039Glu8AAC
1,a40RN0000011frmYAA,105317476,a3yRN0000006s9DYAQ,0018c00002egmwfAAA,105317476,105317476,0038c000039GlttAAC
2,a40RN0000011frnYAA,105692226,a3yRN0000006s9DYAQ,0018c00002egmwtAAA,105692226,105692226,0038c000039Glu7AAC
3,a40RN0000011froYAA,106493316,a3yRN0000006s9DYAQ,0018c00002amTYaAAM,106493316,106493316,0038c000035CdEZAA0
4,a40RN0000011frpYAA,115527029,a3yRN0000006s9DYAQ,0018c00002egmwWAAQ,106493316,115527029,0038c000039GltkAAC


In [ ]:
""" if(campaña!="MetLife"):
    pacientePrograma.to_excel("F:/Empresas SF/reportes/paciente_programas.xlsx", index=False) """

Rescatamos todos los ruts Id con su empresa desde la campaña


In [388]:
Pacientes_Archivo.head()

,rut1,rut2,nombres,Primer apellido,Segundo apellido,apellidos,email,celular,nacimiento,sexo,poliza,empleador,rutE,rut,rutT
0,10063271-3,10063271-3,Marcos Javier,ALARCON,ANDRADE,Alarcon Andrade,marcos.alarcon@instruvalve.cl,(+56) 982893563,1966-06-19,NaN,24277,24277,24277,100632713,100632713
1,17867250-9,17867250-9,Javier Ignacio,ALARCON,SAN MARTIN,Alarcon San Martin,javier.alarcon@instruvalve.cl,(+56) 988276605,1991-04-02,NaN,33330,33330,33330,178672509,178672509
2,19430135-9,19430135-9,Rodrigo Alejandro,ALARCON,SAN MARTIN,Alarcon San Martin,rodrigo.alarcon@instruvalve.com.pe,(+56) 978775171,1996-10-11,NaN,35349,35349,35349,194301359,194301359
3,26945028-2,26945028-2,Jesus Rene,GIBENS,CALANCHE,Gibens Calanche,jesus.gibens@instruvalve.cl,(+56) 933037469,1954-01-07,NaN,19731,19731,19731,269450282,269450282
4,15360659-5,15360659-5,Jimmy Jesus,ORELLANA,CARRASCO,Orellana Carrasco,jimmy.orellana@instruvalve.cl,(+56) 981381146,1984-04-20,NaN,30792,30792,30792,153606595,153606595


Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)


Pacientes_Archivo.drop_duplicates(
[
"rut",
"rutT",
"LLaveEmpresa",
"LLAveRut",
],
keep="last",
)
Pacientes_Archivo.shape


In [389]:
print(campaña)

Instruvalve


In [390]:
Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

In [ ]:
# Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/Pacientes_Archivo.xlsx", index=False)

In [391]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
# Pacientes_con_id.to_excel("F:/Empresas SF/reportes/Pacientes_con_id.xlsx", index=False)
Pacientes_con_id.shape

(91, 26)

In [392]:
Pacientes_con_id.head()

,rut1,rut2,nombres,Primer apellido,Segundo apellido,apellidos,email,celular,nacimiento,sexo,...,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono,Id Contacto
0,10063271-3,10063271-3,Marcos Javier,ALARCON,ANDRADE,Alarcon Andrade,marcos.alarcon@instruvalve.cl,(+56) 982893563,1966-06-19,NaN,...,0018c00002egmwuAAA,100632713,Marcos Javier Alarcon Andrade,RUT,None,None,None,Hombre,None,0038c000039Glu8AAC
1,17867250-9,17867250-9,Javier Ignacio,ALARCON,SAN MARTIN,Alarcon San Martin,javier.alarcon@instruvalve.cl,(+56) 988276605,1991-04-02,NaN,...,0018c00002amRwrAAE,178672509,Javier Ignacio Alarcón San Martin,RUT,None,None,None,Hombre,988276605,0038c000035CbcxAAC
2,19430135-9,19430135-9,Rodrigo Alejandro,ALARCON,SAN MARTIN,Alarcon San Martin,rodrigo.alarcon@instruvalve.com.pe,(+56) 978775171,1996-10-11,NaN,...,0018c00002egmx0AAA,194301359,Rodrigo Alejandro Alarcon San Martin,RUT,None,None,None,Hombre,None,0038c000039GluEAAS
3,26945028-2,26945028-2,Jesus Rene,GIBENS,CALANCHE,Gibens Calanche,jesus.gibens@instruvalve.cl,(+56) 933037469,1954-01-07,NaN,...,0018c00002ehodyAAA,269450282,Jesus Rene Gibens Calanche,RUT,None,None,None,Otros,None,0038c000039HniUAAS
4,15360659-5,15360659-5,Jimmy Jesus,ORELLANA,CARRASCO,Orellana Carrasco,jimmy.orellana@instruvalve.cl,(+56) 981381146,1984-04-20,NaN,...,0018c00002egmwrAAA,153606595,Jimmy Jesus Orellana Carrasco,RUT,None,None,None,Hombre,None,0038c000039Glu5AAC


In [ ]:
# Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)

In [393]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")


def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""


def fechaValida(fecha):
    if (len(str(fecha)) <= 9 ):
        fecha = ""
        
    fecha = str(fecha)
    fecha1 = str(fecha)
    
    #datetime.strptime(fecha, '%Y-%m-%d')
    
    try:
        fecha = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        date_string = fecha1
        print("date_string =", date_string)
        print("type of date_string =", type(date_string))

        date_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")

        print("date_object =", date_object)
        print("type of date_object =", type(date_object))
        return date_object
    except ValueError:
        return datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    

In [ ]:
Pacientes_con_id["dertyuiop"] = Pacientes_con_id["nacimiento"]
Pacientes_con_id["derty"] = Pacientes_con_id["nacimiento"].apply(lambda y: fechaValida(y))
Pacientes_con_id.head()

In [394]:


# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
# Subir_pacientes["Email_Seteado__c"] = ""
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )


if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = Pacientes_con_id["email"].astype(str).str.lower()
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(
        lambda y: isValid(y)
    )
    Subir_pacientes["Email_Seteado__c"] = np.where(
        Subir_pacientes["PersonEmail"] != "", "True", "False"
    )

    # Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
#if ("nacimiento" in Pacientes_con_id.columns) == True:    
#    Subir_pacientes["PersonBirthdate"] = pd.to_datetime(Pacientes_con_id["nacimiento"].astype(str), format="%Y-%m-%d")  
if ("nacimiento" in Pacientes_con_id.columns) == True:    
    #try:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"].apply(lambda y: fechaValida(y))
    Subir_pacientes["derty"] = Pacientes_con_id["nacimiento"].apply(lambda y: fechaValida(y))
    #except ValueError:
        #Subir_pacientes["PersonBirthdate"] = pd.to_datetime("1900-01-01", format="%Y-%m-%d")
 
if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "H": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "Prefiero no responder",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "M": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon": "Hombre",
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace(".", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("/", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("null", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NULL", "")


Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"

Subir_pacientes_Que_no_Estan = Subir_pacientes
Subir_pacientes_Estan = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == True]
Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]

#Subir_pacientes["PersonBirthdate"] = Subir_pacientes["PersonBirthdate"].str.replace("0001-01-01 00:00:00", "")

# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel(
    "F:/Empresas SF/reportes/Pacientes_Que_No_Estaban.xlsx", index=False
)
Subir_pacientes.head()

date_string = 1966-06-19 00:00:00
type of date_string = <class 'str'>
date_object = 1966-06-19 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1991-04-02 00:00:00
type of date_string = <class 'str'>
date_object = 1991-04-02 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1996-10-11 00:00:00
type of date_string = <class 'str'>
date_object = 1996-10-11 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1954-01-07 00:00:00
type of date_string = <class 'str'>
date_object = 1954-01-07 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1984-04-20 00:00:00
type of date_string = <class 'str'>
date_object = 1984-04-20 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1989-12-27 00:00:00
type of date_string = <class 'str'>
date_object = 1989-12-27 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1971-05-24 00:00:00
type of date_string = <class 'str'>
date_o

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\1446579255.py:94: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\1446579255.py:101: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")


,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,Email_Seteado__c,PersonBirthdate,derty,HealthCloudGA__Gender__pc,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
9,False,Carlos Martin,Curiqueo Traipe,142702576,carlosmartincuriqueo@gmail.com,True,1970-06-08,1970-06-08,NaN,(56) 934425137,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
26,False,Andres Ignacio,Zavala Sepulveda,187955378,andres.zavala@instruvalve.cl,True,1994-06-22,1994-06-22,NaN,(56) 936569573,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
31,False,Ricardo Antonio,Abarca Leiva,153968225,rabarca@vlas.cl,True,1982-01-09,1982-01-09,NaN,(56) 998353582,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
54,False,Moises Jonathan,Miranda Peralta,172046347,moises.miranda@instruvalve.com,True,1989-07-28,1989-07-28,NaN,(56) 98397643,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
67,False,Manuel Alejandro,Salgado Diaz,170297202,manuel.salgado@instruvalve.com,True,1989-03-11,1989-03-11,NaN,(56) 99291334,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [395]:
Subir_pacientes.shape

(21, 15)

In [396]:
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc)"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return "None"


def traerContact(ide):
    ide = str(ide)
    if len(ide) >= 12:
        idcontact = sf.Account.get(ide)
        return idcontact["PersonContactId"]
    else:
        return "NONES"


def agregarPaciente(listado):
    vListado = listado.split(",")
    
    derty = None
    
        
        
    if (vListado[9] == ""):                               
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],  
                                          
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    else:
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],
                        "PersonBirthdate": vListado[9],
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    # Verify that the Contact was created successfully
    # created_contact = sf.Contact.get(sf_res['id'])
    # print(created_contact)


""" sf_res =None
try:
    # Create the new Contact record in Salesforce
    sf_res = sf.Contact.create(new_contact)
except SalesforceMalformedRequest as e:
    if 'duplicateResut' in e[0]:
        print(f"Malformed request error: {e}")
        # Handle the error here as needed
        #Bypass SF Email duplicate rule by removing the email to create a SF Lead
        del new_contact['Email']
        sf_res = sf.Contact.create(new_contact)


if sf_res is not None:
    # Verify that the Contact was created successfully
    created_contact = sf.Contact.get(sf_res['id'])
    print(created_contact) """

' sf_res =None\ntry:\n    # Create the new Contact record in Salesforce\n    sf_res = sf.Contact.create(new_contact)\nexcept SalesforceMalformedRequest as e:\n    if \'duplicateResut\' in e[0]:\n        print(f"Malformed request error: {e}")\n        # Handle the error here as needed\n        #Bypass SF Email duplicate rule by removing the email to create a SF Lead\n        del new_contact[\'Email\']\n        sf_res = sf.Contact.create(new_contact)\n\n\nif sf_res is not None:\n    # Verify that the Contact was created successfully\n    created_contact = sf.Contact.get(sf_res[\'id\'])\n    print(created_contact) '

In [397]:
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,Email_Seteado__c,PersonBirthdate,derty,HealthCloudGA__Gender__pc,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
9,False,Carlos Martin,Curiqueo Traipe,142702576,carlosmartincuriqueo@gmail.com,True,1970-06-08,1970-06-08,NaN,(56) 934425137,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
26,False,Andres Ignacio,Zavala Sepulveda,187955378,andres.zavala@instruvalve.cl,True,1994-06-22,1994-06-22,NaN,(56) 936569573,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
31,False,Ricardo Antonio,Abarca Leiva,153968225,rabarca@vlas.cl,True,1982-01-09,1982-01-09,NaN,(56) 998353582,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
54,False,Moises Jonathan,Miranda Peralta,172046347,moises.miranda@instruvalve.com,True,1989-07-28,1989-07-28,NaN,(56) 98397643,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
67,False,Manuel Alejandro,Salgado Diaz,170297202,manuel.salgado@instruvalve.com,True,1989-03-11,1989-03-11,NaN,(56) 99291334,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [398]:
Subir_pacientes.drop_duplicates(
    [
        "IdentificationId__pc",
    ],
    keep="last",
)

Subir_pacientes["Id"] = Subir_pacientes["IdentificationId__pc"].apply(
    lambda y: traerId(y)
)

Subir_pacientes.shape

001RN000006B88ZYAS 142702576
001RN000006AyvdYAC 187955378
001RN000006B6d5YAC 153968225
001RN000006B88aYAC 172046347
001RN000006B1zzYAC 170297202
001RN000006B9fhYAC 225463654
001RN000006B9UOYA0 214702258
001RN000006BA74YAG 228940887
001RN000006AuiFYAS 279374819
001RN000006BBsjYAG 154516751
001RN000006BBuLYAW 23116779K
001RN000006BBuLYAW 23116779K
001RN000006B6ebYAC 222919177
001RN000006BABwYAO 112292241
001RN000006B1myYAC 23705976K
001RN000006BBUZYA4 228503479
001RN000006B6wJYAS 266128266
001RN000006BAJyYAO 266936117
001RN000006B24vYAC 275357499
001RN000006BBvxYAG 1891078978
001RN000006AwlhYAC 275357944


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\2950418267.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Subir_pacientes["Id"] = Subir_pacientes["IdentificationId__pc"].apply(


(21, 16)

In [399]:
Subir_pacientes["Id"] = Subir_pacientes["Id"].astype(str)
Subir_pacientes = Subir_pacientes[Subir_pacientes["Id"].str.len() <= 12]
# Subir_pacientes["numero"] = Subir_pacientes["Id"].str.len()
Subir_pacientes.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\2892125083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Subir_pacientes["Id"] = Subir_pacientes["Id"].astype(str)


(0, 16)

In [ ]:
#Subir_pacientes["dato_account"].head()

In [ ]:
Subir_pacientes.to_excel("F:/Empresas SF/reportes/Todesffff.xlsx", index=False)

In [400]:
Subir_pacientes["dato_account"] = ""
if ("FirstName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes[
        "FirstName"
    ].astype(str)
if ("LastName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["LastName"].astype(str)
    )
if ("IdentificationId__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["IdentificationId__pc"].astype(str)
    )
if ("RecordTypeId" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["RecordTypeId"].astype(str)
    )
if ("BillingCountry" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCountry"].astype(str)
    )
if ("Identification_Type__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Identification_Type__c"].astype(str)
    )
if ("Pa_s_del_Paciente__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Pa_s_del_Paciente__c"].astype(str)
    )
if ("Cuenta_validada__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Cuenta_validada__c"].astype(str)
    )

if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["PersonEmail"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

if ("PersonBirthdate" in Subir_pacientes.columns) == True:
    #if Subir_pacientes["dato_account"] != "":
        #fecha = pd.to_datetime(Subir_pacientes["PersonBirthdate"].astype(str) + " 00:00:00").isoformat() + "Z" 
        #fecha = pd.to_datetime(Subir_pacientes["PersonBirthdate"].astype(str) + " 00:00:00")                       
        Subir_pacientes["dato_account"] = (Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonBirthdate"].astype(str)
            
        )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("HealthCloudGA__Gender__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["HealthCloudGA__Gender__pc"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("Phone" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["Phone"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingCity" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCity"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingState" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingState"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingStreet" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingStreet"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Email_Seteado__c"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

# Subir_pacientes["dato_account"] =  Subir_pacientes["dato_account"] =

Subir_pacientes["Id_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
)
""" Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
) """

' Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(\n    lambda x: agregarPaciente(x)\n) '

In [401]:
query2 = "select P_liza__c, Nombre_de_Empresa__c,Identificador__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c WHERE Empresa_Padre__r.Name = '" + campaña + "'"

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]
df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_asociadas.xlsx", index=False)
df2.head()

100%|██████████| 1/1 [00:02<00:00,  2.29s/it]


,P_liza__c,Nombre_de_Empresa__c,Identificador__c,Rut_Empresa__c,Empresa_Padre__r.Name,Holding__c,Plan__c,Pais__c,Activo__c,Id,LLave,dato
0,-,Instruvalve S.A.,Instruvalve S.A.,-,Instruvalve,-,Plan Silver,Chile,True,a3yRN0000006s9DYAQ,-,1
1,-,Instruvalve SPA,Instruvalve SPA,-,Instruvalve,-,Plan Silver,Chile,True,a3yRN0000006s9EYAQ,-,1


In [402]:
Pacientes_con_id_no = Pacientes_con_id[Pacientes_con_id["RutEstaoNo"] == False]
Pacientes_con_id_no["Id Account Paciente"] = Pacientes_con_id_no["rut"].apply(
    lambda y: traerId(y)
)
Pacientes_con_id_no["Id Contacto"] = Pacientes_con_id_no["Id Account Paciente"].apply(
    lambda y: traerContact(y)
)
Pacientes_con_id_no.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_no.xlsx", index=False
)

Pacientes_con_id_si = Pacientes_con_id[Pacientes_con_id["RutEstaoNo"] == True]
Pacientes_con_id_si.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_si.xlsx", index=False
)

Pacientes_con_id_final = pd.concat(
    [
        Pacientes_con_id_si,
        Pacientes_con_id_no,
    ],
    axis=0,
)

Pacientes_con_id_final.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_final.xlsx", index=False
)

if campaña == "Southbridge Plan Mujer":
    VpacientePrograma = "a1WRN0000007YwD2AU"
    VempresaaAccount = "001RN000003rwVaYAI"
    VempresaConvenio = "a3yRN0000006sVTYAY"
elif campaña == "Southbridge":
    VpacientePrograma = "a1W8c0000083pM9EAI"
    VempresaaAccount = "0018c00002f3Ev2AAE"
    VempresaConvenio = "a3yRN0000006sqYYAQ"
elif campaña == "Construmart":
    VpacientePrograma = "a1W8c0000083q9TEAQ"
    VempresaaAccount = "0018c00002ghJgBAAU"
    VempresaConvenio = "a3yRN0000006rlZYAQ"
elif campaña == "EY":
    VpacientePrograma = "a1W8c0000083laZEAQ"
    VempresaaAccount = "0018c00002eTboVAAS"
    VempresaConvenio = "a3yRN0000006rsTYAQ"
elif campaña == "Colmena":
    VpacientePrograma = "a1W8c0000083j8rEAA"
    VempresaaAccount = "0018c00002cXclQAAS"
    VempresaConvenio = "a3yRN0000006rf5YAA"
elif campaña == "Pedro De Valdivia":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006sVzYAI"
elif campaña == "Forum":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006s3SYAQ"
elif campaña == "Cruz Verde":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006rsqYAA"
elif campaña == "Milan Fajanovic":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006sR4YAI"
elif campaña == "ProChile":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006rpsYAA"
elif campaña == "Farmacias Ahumada":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006rspYAA"
elif campaña == "Minera El Abra":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006snOYAQ"
elif campaña == "Instruvalve":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006s9DYAQ"
elif campaña == "VidaCel":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006suzYAA"
elif campaña == "Care Assistance Colaboradores":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006reXYA"
elif campaña == "Nexo":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006sORYAY"
elif campaña == "Talo":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006t10YAA"
else:
    VpacientePrograma = "Buscarv"
    VempresaaAccount = "Buscarv"
    VempresaConvenio = "Buscarv"

#Programas por empresas
Pacientes_con_id_final["esonoes"] = Pacientes_con_id_final["rut"].isin(
    PacientesProgramas_PacientesSF["Rut_y"]
)
Subir_programas_empresa = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes"] == False
]
Subir_programas_empresa["Paciente__c"] = Subir_programas_empresa["Id Account Paciente"]
Subir_programas_empresa["Programas_por_Empresas__c"] = VpacientePrograma
Subir_programas_empresa = Subir_programas_empresa[
    [
        # "empresa",
        "poliza",
        "Paciente__c",
        "Programas_por_Empresas__c",
    ]
]
Subir_programas_empresa.to_excel(
    "F:/Empresas SF/reportes/programas_empresa.xlsx", index=False
)

#
Pacientes_con_id_final["esonoes1"] = Pacientes_con_id_final["rut"].isin(
    AccountContact_PacientesSF["Rut_y"]
)
Subir_AccountContact = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes1"] == False
]
Subir_AccountContact["ContactId"] = Subir_AccountContact["Id Contacto"]
Subir_AccountContact["AccountId"] = VempresaaAccount

Subir_AccountContact = Subir_AccountContact[
    [
        # "empresa",
        "poliza",
        "ContactId",
        "AccountId",
    ]
]
Subir_AccountContact.to_excel(
    "F:/Empresas SF/reportes/account_Contact.xlsx", index=False
)

empresacontactoconvenio_PacientesSF["llave"] = empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_y"]
Pacientes_con_id_final["llaveA"] = Pacientes_con_id_final["rutT"] + Pacientes_con_id_final["rut"]

Pacientes_con_id_final["esonoes2"] = Pacientes_con_id_final["llaveA"].isin(
    empresacontactoconvenio_PacientesSF["llave"]
)
Subir_empresacontactoconvenio = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes2"] == False
]
Subir_empresacontactoconvenio["Paciente__c"] = Subir_empresacontactoconvenio[
    "Id Account Paciente"
]
Subir_empresacontactoconvenio["N_Documento_Titular__c"] = Subir_empresacontactoconvenio[
    "rutT"
]
Subir_empresacontactoconvenio["Empresa__c"] = VempresaConvenio

Subir_empresacontactoconvenio = Subir_empresacontactoconvenio[
    [
        "empleador",
        "rutE",
        "poliza",
        "Paciente__c",
        "N_Documento_Titular__c",
        "Empresa__c",
    ]
]
Subir_empresacontactoconvenio.to_excel(
    "F:/Empresas SF/reportes/empresacontactoconvenio.xlsx", index=False)
Pacientes_con_id_final.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_final2.xlsx", index=False
)


001RN000006B88ZYAS 142702576
001RN000006AyvdYAC 187955378
001RN000006B6d5YAC 153968225
001RN000006B88aYAC 172046347
001RN000006B1zzYAC 170297202
001RN000006B9fhYAC 225463654
001RN000006B9UOYA0 214702258
001RN000006BA74YAG 228940887
001RN000006AuiFYAS 279374819
001RN000006BBsjYAG 154516751
001RN000006BBuLYAW 23116779K
001RN000006BBuLYAW 23116779K
001RN000006B6ebYAC 222919177
001RN000006BABwYAO 112292241
001RN000006B1myYAC 23705976K
001RN000006BBUZYA4 228503479
001RN000006B6wJYAS 266128266
001RN000006BAJyYAO 266936117
001RN000006B24vYAC 275357499
001RN000006BBvxYAG 1891078978
001RN000006AwlhYAC 275357944


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\1369155135.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pacientes_con_id_no["Id Account Paciente"] = Pacientes_con_id_no["rut"].apply(
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\1369155135.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pacientes_con_id_no["Id Contacto"] = Pacientes_con_id_no["Id Account Paciente"].apply(
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18988\1369155135.py:109: SettingWithCopyWarning: 
A value is try

In [ ]:
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "163820064")].index, inplace=True)
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "153298939")].index, inplace=True)
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "191710940")].index, inplace=True)
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "182470082")].index, inplace=True)
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "16277865")].index, inplace=True)
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "16277865K")].index, inplace=True)
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "HEGJ930928HMNRRL06")].index, inplace=True)
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "16007258K")].index, inplace=True)
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "86676540")].index, inplace=True)
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "202201466")].index, inplace=True)
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "105233469")].index, inplace=True)
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "130949185")].index, inplace=True)
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "255356798")].index, inplace=True)
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "34005945")].index, inplace=True)
empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "185410072")].index, inplace=True)



In [ ]:
empresacontactoconvenio_PacientesSF.to_excel(
    "F:/Empresas SF/reportes/SFpacientes.xlsx", index=False)

In [ ]:
contactd = sf.Account.get("0018c00002lUyo0AAC")
contactd["PersonContactId"]

In [ ]:
# contactd["PersonContactId"]
print(datetime.now())

In [ ]:
Subir_pacientes.to_excel("F:/Empresas SF/reportes/todes.xlsx", index=False)

query2 = "select P_liza**c, Nombre_de_Empresa**c, Rut_Empresa**c, Empresa_Padre**r.Name, Empresa**r.name,Holding**c, Plan**c, Pais**c, Activo**c, Id from Datos_Empresa**c "

df2 = sf.smart_query(query2)
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()


Subir_pacientes.to_excel("F:/Empresas SF/reportes/111account.xlsx", index=False)


derty = Subir_pacientes["IdentificationId__pc"]
sf.Contact.create({"IdentificationId\_\_pc": "derty", "Email": "example@example.com"})


Subir_pacientes.head()


Subir_pacientes.head()
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account\_\_l.xlsx", index=False)


In [ ]:
""" if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )
 """

In [ ]:
""" if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    ) """

In [ ]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [ ]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)

In [ ]:
campaña_en_pacientes2["rut"].shape

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
rutes = pd.DataFrame(columns=["rut"])
rutes["rut"] = Pacientes_Archivo["rut"]
rutes["rutT"] = Pacientes_Archivo["rut2"]
rutes["Contratante"] = Pacientes_Archivo["Contratante"]
# rutes["poliza"] = Pacientes_Archivo["poliza"]
rutes.head()

In [ ]:
rutes["rut"] = rutes["rut"].astype(str)
rutes["rut"] = rutes["rut"].str.replace(".", "")
rutes["rut"] = rutes["rut"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rut"] = rutes["rut"].str.upper()
rutes["rut"] = rutes["rut"].str.strip()

rutes["rutT"] = rutes["rutT"].astype(str)
rutes["rutT"] = rutes["rutT"].str.replace(".", "")
rutes["rutT"] = rutes["rutT"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rutT"] = rutes["rutT"].str.upper()
rutes["rutT"] = rutes["rutT"].str.strip()

""" rutes["poliza"] = rutes["poliza"].astype(str)
rutes["poliza"] = rutes["poliza"].str.replace(".", "")
rutes["poliza"] = rutes["poliza"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["poliza"] = rutes["poliza"].str.strip() """


rutes.head()

In [ ]:
# derty = rutes.to_numpy().transpose().tolist()


Pacientes_SF2 = Pacientes_SF[["Rut", "Id Account Paciente"]]

In [ ]:
rutesi = pd.merge(
    left=rutes,
    right=Pacientes_SF2,
    how="left",
    left_on="rut",
    right_on="Rut",
)
rutesi.shape
rutesi.to_excel("F:/Empresas SF/reportes/rutsyempresa.xlsx", index=False)

In [ ]:
rutesi.head()

In [ ]:
rutesi_antg = pd.merge(
    left=rutesi,
    right=empresas_unidas,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)

In [ ]:
rutesi_antg.head()

In [ ]:
# rutesi_antg .to_excel("F:/Empresas SF/reportes/ruts_empresas_antiguos.xlsx", index=False)

In [ ]:
import json
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps
import simpler_sf

simpler_sf.simple_salesforce()

In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [ ]:
""" query = "select Id, Account.Name, RutContact__c from AccountContactRelation "

df1 = sf.smart_query(query) """

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
# rutes = Subir_pacientes["rut"]

In [ ]:
""" df1.head() """

In [ ]:
""" df1["rut"] = df1["RutContact__c"].astype(str)
df1["rut"] = df1["rut"].str.replace(".", "")
df1["rut"] = df1["rut"].str.replace(
    "-", ""
)  # aca se trabaja con el nuevo rut desde rut1 ojooooooo
df1["rut"] = df1["rut"].str.upper()
df1["rut"] = df1["rut"].str.strip()
df1.head() """

In [ ]:
""" rutes.head() """

In [ ]:
""" rutesi = pd.merge(
    left=rutes,
    right=df1,
    how="left",
    left_on="rut",
    right_on="rut",
) """

In [ ]:
""" rutesi.to_excel("F:/Empresas SF/reportes/rutesnuevos_accountcontactrelation.xlsx", index=False) """

In [ ]:
""" df2["P_liza__c"] = df2["P_liza__c"].astype(str)
df2["P_liza__c"] = df2["P_liza__c"].str.replace(".", "")
df2["P_liza__c"] = df2["P_liza__c"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
df2["P_liza__c"] = df2["P_liza__c"].str.upper()
df2["P_liza__c"] = df2["P_liza__c"].str.strip() """

In [ ]:
modelo_nuevo = pd.merge(
    left=rutesi,
    right=df2,
    how="left",
    left_on="poliza",
    right_on="P_liza__c",
)
modelo_nuevo.to_excel("F:/Empresas SF/reportes/modelo_nuevo.xlsx", index=False)

In [ ]:
modelo = pd.DataFrame()
modelo = modelo_nuevo[["rutT", "Id Account Paciente", "Id"]]

In [ ]:
modelo.head()